# Superstore EDA

**Notebook:** `Superstore_EDA.ipynb`

This notebook performs Exploratory Data Analysis (EDA) on the `Superstore.csv` dataset. **Note:** The dataset `Superstore.csv` is NOT included in the ZIP. Place the dataset in the same folder as this notebook before running.

---

**Notebook contents:** Business goals, data inspection, missing data handling strategy, summary statistics, visualizations, outlier detection, and final recommendations.

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from scipy import stats
from sklearn.impute import SimpleImputer

print("Imports done")

In [ ]:
# Dataset path - put Superstore.csv in the same folder as this notebook
data_path = "Superstore.csv"
import os
if not os.path.exists(data_path):
    raise FileNotFoundError(f"Dataset not found at {data_path}. Please place 'Superstore.csv' in this folder.")
else:
    print("Dataset found:", data_path)

In [ ]:
# Load dataset and preview
df = pd.read_csv(data_path)
print("Shape:", df.shape)
display(df.head(8))
display(df.dtypes)

## Business Understanding & Goals

**Objective:** Analyze Superstore sales data to uncover trends, customer/product segments, and factors affecting sales and profit. This will help guide decisions to optimize inventory, pricing, promotions, and target marketing.

**Research question / business goal:** What are the main drivers of profit and which product categories, regions, or customer segments are most valuable? How can we improve overall profitability?

**Target variables / key variables:**
- Target variable (for modeling): `Profit` or a derived classification like `High vs Low Profit`.
- Key variables: `Sales`, `Quantity`, `Discount`, `Profit`, `Category`, `Sub-Category`, `Region`, `Segment`, `Order Date`, `Ship Mode`.

**KPI to measure success:**
- Increase in average profit per order or increase in overall profit margin.
- Metrics: mean Profit, total Sales, profit margin (Profit/Sales).

In [ ]:
# Data inspection
print("Rows, columns:", df.shape)
print("\nMissing values per column:\n", df.isna().sum().sort_values(ascending=False).head(20))
print("\nNumber of duplicate rows:", df.duplicated().sum())
print("\nBasic descriptive statistics (numeric):")
display(df.describe().transpose())

In [ ]:
# Missing data heatmap (requires seaborn)
import matplotlib.pyplot as plt
plt.figure(figsize=(10,6))
sns.heatmap(df.isna(), cbar=False)
plt.title('Missing values heatmap')
plt.show()

### Data types and date parsing
- Convert `Order Date` and `Ship Date` to datetime if present and extract `year`, `month`, `weekday` features for trend analysis.


## Handling Missing Data

For each variable with missing data, document whether to drop, impute, or leave as-is. Use SimpleImputer for numeric (median) and mode for categorical. Provide justification in the notebook and save a table summarizing decisions.


In [ ]:
# Example missing handling (do not change without checking)
missing = df.isna().sum()
missing = missing[missing>0]
missing_table = []
for col in missing.index:
    dtype = df[col].dtype
    if pd.api.types.is_numeric_dtype(df[col]):
        method = "median imputation (SimpleImputer)"
    else:
        method = "mode imputation"
    missing_table.append({"column": col, "missing_count": int(missing[col]), "dtype": str(dtype), "suggested_method": method})

display(pd.DataFrame(missing_table))

## Summary Statistics & Visual Exploration
- Compute central tendency, spread, skewness, kurtosis for numeric variables.
- Visualize histograms, boxplots, QQ plots, and identify outliers using IQR and Z-score.


In [ ]:
# Numeric summary with skewness/kurtosis
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
stats_df = df[num_cols].agg(['mean','median','std','min','max','skew','kurt']).transpose().reset_index()
stats_df = stats_df.rename(columns={'index':'column'})
display(stats_df)

In [ ]:
# Histograms and boxplots for key numeric cols
import matplotlib.pyplot as plt
for col in ['Sales','Quantity','Discount','Profit']:
    if col in df.columns:
        plt.figure(figsize=(12,4))
        plt.subplot(1,2,1)
        sns.histplot(df[col].dropna(), kde=True)
        plt.title(f'Histogram of {col}')
        plt.subplot(1,2,2)
        sns.boxplot(x=df[col].dropna())
        plt.title(f'Boxplot of {col}')
        plt.show()

## Outlier Detection
- Use IQR and Z-score methods to detect outliers on numeric variables.
- Decide whether to remove/winsorize/keep based on domain knowledge.


In [ ]:
# Example: IQR outlier detection for Sales and Profit
outlier_info = {}
for col in ['Sales','Profit']:
    if col in df.columns and pd.api.types.is_numeric_dtype(df[col]):
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        outliers = df[(df[col] < lower) | (df[col] > upper)][col]
        outlier_info[col] = {'lower': lower, 'upper': upper, 'n_outliers': outliers.shape[0]}
outlier_info

## Relationships & Correlations
- Visualize correlation matrix for numeric variables and pairwise plots for selected features.


In [ ]:
# Correlation heatmap
plt.figure(figsize=(10,8))
sns.heatmap(df.select_dtypes(include=[np.number]).corr(), annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Correlation matrix (numeric features)')
plt.show()

## Categorical variables
- Show frequency tables and bar plots for Category, Sub-Category, Region, Segment, Ship Mode.


In [ ]:
# Categorical distributions - top categories
cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()
for col in ['Category','Sub-Category','Region','Segment','Ship Mode']:
    if col in df.columns:
        print('\n',col, 'value counts:')
        display(df[col].value_counts().head(10))
        plt.figure(figsize=(6,3))
        sns.countplot(y=col, data=df, order=df[col].value_counts().index)
        plt.title(f'Count of {col}')
        plt.show()

## Final Recommendations & Next Steps
- Summarize key findings and recommendations for business.
- Suggest modeling steps (feature engineering, target creation, train/validation strategy).


In [ ]:
# Save summary outputs (already computed by helper script)
print("Summary files included with notebook (not dataset):")
print("- eda_results_summary.csv")
print("- eda_report.txt")

---
**Notes:**
- This notebook is designed to be reproducible. Place `Superstore.csv` in the same folder and run all cells.
- The ZIP for submission will include the notebook and a short report/summary CSV but NOT the dataset.
